In [3]:
import pandas as pd
from lxml import html
from lxml import etree
import requests
import json
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import time
import re
import numpy as np
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from pprint import pprint
import pickle
import copy

In [47]:
url = 'https://www.capitaliq.com/ciqdotnet/login-sso.aspx'
driver = webdriver.Chrome(executable_path=r'/Users/archiefox/Desktop/chromedriver')
driver.get(url)


In [20]:

# login

# pickle.dump(driver.get_cookies(), open('mac_cookies.pkl', 'wb'))

In [48]:
cookies = pickle.load(open('mac_cookies.pkl', 'rb'))
for cookie in cookies:
    driver.add_cookie(cookie)

In [49]:
driver.get('https://www.capitaliq.com/CIQDotNet/my/dashboard.aspx')

In [50]:
search_bar = driver.find_element_by_name("SearchText")

In [51]:
name = 'Hammertech Global'
search_bar.send_keys(name)
search_bar.send_keys(Keys.ENTER)

In [52]:
def scrape_iq_data(driver):
    data = {}
    revenue = driver.find_element_by_xpath('//td//*[contains(text(), "Total Revenue")]').find_element_by_xpath('..//following-sibling::td').text
    revenue = float(revenue.replace(",", ""))*(10**6)
    data['Revenue'] = revenue
    
    ebitda = driver.find_element_by_xpath('//td//*[contains(text(), "EBITDA")]').find_element_by_xpath('..//following-sibling::td').text
    ebitda = float(ebitda.replace(",", ""))*(10**6)
    data['EBITDA'] = ebitda
    
    d = pd.read_html(driver.page_source)
    for df in d[::-1]:
        if 'Key Professionals View All' in df.values:
            execs = df.rename(columns=df.iloc[1]).drop(df.index[:2]).reset_index(drop=True).iloc[:, :2]
            execs['Title'] = execs['Title'].str.upper().str.replace(',', ' ')
            execs = execs.astype(str)
            try:
                ceo = execs[execs.Title.apply(lambda x: any(identifier in x for identifier in ['CEO', 'CHIEF EXECUTIVE OFFICER', "CHIEF EXEC. OFFICER"]))].Name.values[0]
                data['CEO'] = ceo
                print(ceo)
            except:
                print('ceo not found')
            try:
                cfo = execs[execs.Title.apply(lambda x: any(identifier in x for identifier in ['CFO', 'CHIEF FINANCIAL OFFICER']))].Name.values[0]
                data['CFO'] = cfo
                print(cfo)
            except:
                print('cfo not found')
            try:
                cto = execs[execs.Title.apply(lambda x: any(identifier in x.replace('DIRECTOR', '') for identifier in ['CTO', 'CHIEF TECHNOLOGY OFFICER']))].Name.values[0]
                data['CTO'] = cto
                print(cto)
            except:
                print('cto not found')
            return data

In [53]:
d = pd.read_html(driver.page_source)
for df in d[::-1]:
    if 'Key Professionals View All' in df.values:
        execs = df.rename(columns=df.iloc[1]).drop(df.index[:2]).reset_index(drop=True).iloc[:, :2]
        execs = execs.astype(str)
        display(execs)
        break

,Name,Title
0,"Leach, Ben",Co-Founder & CEO
1,"Harris, James",Co-Founder & CTO
2,"Hogben, Andrew",Co-Founder & VP of sales - North America
3,"McDonald, Lucas",Co-Founder
4,"Tabone, Brad",Co-Founder & COO
5,"Stassen, Wynand",CFO & VP of Corporate Services
6,"Lucett, Virginia Chere",Vice President of Global Marketing
7,"Hobbs, Adam",Head of Sales (APAC)
8,"Peters, Nicholas",Head of Marketing (APAC)
9,nan,nan


In [45]:
execs.Title.apply(lambda x: any(identifier in x for identifier in ['CEO', 'CHIEF EXECUTIVE OFFICER', "CHIEF EXEC. OFFICER"]))

0      True
1     False
2      True
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
Name: Title, dtype: bool

In [46]:
scrape_iq_data(driver)

Lombardo, Anthony Peter
Dixon, Simon Collier
cto not found


{'Revenue': 9892000000.0,
 'EBITDA': 126000000.0,
 'CEO': 'Lombardo, Anthony Peter',
 'CFO': 'Dixon, Simon Collier'}

In [ ]:
if 'Australia' in address:
    if 'Fax' in phone_fax:
        phone, fax = [re.sub("[^0-9]", "", i) for i in phone_fax.split('Fax:')]
    else:
        phone = re.sub("[^0-9]", "", phone_fax)

In [39]:
execs = execs.as

,Name,Title
0,"Lombardo, Anthony Peter","Global CEO, MD & Executive Director"
1,"Dixon, Simon Collier",Group Chief Financial Officer
2,"Hickey, Denis Jon",Global COO & CEO of Lendlease Americas
3,"Rampa, Kylie K.",Group Head of Investments
4,"McCarthy, Justin",Head of Investor Relations
5,"Pedersen, Karen",Group General Counsel
6,"Fadahunsi, Ola",Director of Corporate Communications
7,"Hutton, David Stewart",Group Head of Development
8,"Megale, Luisa",Group Head of Corporate Affairs & Marketing
9,"Yates, Deborah",Chief People Officer


In [42]:
ceo = execs[execs.Title.apply(lambda x: any(identifier in x for identifier in ['CEO', 'CHIEF EXECUTIVE OFFICER', "CHIEF EXEC. OFFICER"]))].Name.values[0]
ceo

'Lombardo, Anthony Peter'